In [2]:
import pandas as pd
import numpy as np
import collections
import copy
import random
import matplotlib.pyplot as plt
import time
import multiprocessing
import nltk
import pickle
import json
from item.item_list import (
    ItemList
)
from nlp.word_embeddings import (
    load_word_embeddings,
    get_items_embeddings
)
from nlp.pos_tagging import (
    get_tokens_tags
)
from item.clustering.item_representation import (
    get_group_embeddings_matrix,
    get_group_embeddings_from_dict,
    save_items_embeddings,
    load_items_embeddings,
    normalize
)
from item.utils import (
    get_tokens_set,
    translate_id_to_descriptions
)
from item.clustering.utils import (
    save_clustering_results_pickle,
    save_models_pickle
)
from item.clustering.clustering import run_baseline_clustering

# Load items

In [3]:
# It gets the descriptions processed:
itemlist = ItemList()
itemlist.load_items_from_file('../data/items_preprocessed_complete_druid.csv.zip')

In [4]:
itemlist.items_df.head(10)

,palavras,unidades_medida,numeros,cores,materiais,tamanho,quantidade,preco,dsc_unidade_medida,original,original_prep,ano,licitacao,licitacao_item,item_id
0,"['encunhamento', 'aperto', 'alvenaria', 'tijol...","['x', 'cm', 'c']","['2.00e+00', '4.00e+00', '1.00e+01', '2.00e+01...",[],[],[],[],15.87,metro,5.2.4 ENCUNHAMENTO (APERTO DE ALVENARIA) EM TI...,"['2', '4', 'encunhamento', 'aperto', 'alvenari...",2017,297095,10000108,0
1,"['piso', 'cimentado', 'desempenar', 'com', 'ac...","['cm', 'm']","['1.00e+00', '3.00e+00', '0.00e+00', '2.00e+00...",[],[],[],[],42.78,metro quadrado,10.1.3 PISO CIMENTADO DESEMPENADO COM ACABAMEN...,"['1', '3', 'piso', 'cimentado', 'desempenar', ...",2017,297095,10000172,1
2,"['chuveiro', 'maxi', 'ducha', 'lorenzetti', 'c...",[],"['8.01e+03', '1.51e+01']",[],['plastico'],[],[],58.78,unidade,"15.12 CHUVEIRO MAXI DUCHA, LORENZETTI,COM MANG...","['chuveiro', 'maxi', 'ducha', 'lorenzetti', 'c...",2017,297095,10000304,2
3,"['tubo', 'soldavel', 'inclusive', 'conexao']",['mm'],"['2.50e+01', '1.91e+01']",[],['pvc'],[],[],3.42,metro,"19.1 TUBO PVC SOLDAVEL 25MM, INCLUSIVE CONEXOES","['tubo', 'pvc', 'soldavel', '25', 'mm', 'inclu...",2017,297095,10000454,3
4,"['tomado', 'modular', 'rj', 'categoria', 'comp...",[],"['4.00e+00', '1.00e+00', '4.50e+01', '6.00e+00...",[],[],[],[],12.37,unidade,20.4.1 TOMADA MODULAR RJ-45 CATEGORIA 6 (COMPL...,"['4', '1', 'tomado', 'modular', 'rj', '45', 'c...",2017,297095,10000472,4
5,"['joelho', 'soldavel', 'fornecimento', 'instal...",['mm'],"['1.00e+00', '4.00e+01', '9.00e+01', '7.50e+01...",[],[],[],[],76.79,unidade,"12.1.40 JOELHO 90 SOLDAVEL - 75MM, FORNECIMENT...","['1', '40', 'joelho', '90', 'soldavel', '75', ...",2017,297095,10000528,5
6,"['aditivo', 'para', 'radiador']",[],[],[],[],[],[],10.80,unidade,ADITIVO PARA RADIADOR,"['aditivo', 'para', 'radiador']",2017,297098,10000591,6
7,"['filtro', 'ar', 'primario', 'ars']",[],['1.01e+03'],[],[],[],[],60.90,unidade,FILTRO DE AR PRIMARIO ARS 1013,"['filtro', 'ar', 'primario', 'ars', '1013']",2017,297098,10000670,7
8,"['cone', 'guta', 'percha', 'acessorio', 'r8', ...",[],['1.20e+02'],[],[],[],"['caixa', 'unidade']",18.80,caixa,CONE DE GUTA PERCHA ACESSORIO R8 (FF) CAIXA CO...,"['cone', 'guta', 'percha', 'acessorio', 'r8', ...",2017,297101,10000720,8
9,"['acido', 'fosforico', 'consistencia', 'gel', ...","['c', '%']","['3.70e+01', '2.50e+00']",[],[],[],[],1.03,unidade,ACIDO FOSFORICO A 37% NA CONSISTENCIA DE GEL S...,"['acido', 'fosforico', '37', 'consistencia', '...",2017,297101,10000723,9


In [5]:
len(itemlist.items_df)

10902496

In [6]:
# Get the tags of tokens descriptions
word_class = get_tokens_tags(itemlist.unique_words)

In [ ]:
descriptions = []
count = 0

for i in range(len(itemlist.items_df)):
    item_dict = itemlist.items_df.loc[i].to_dict()
    original_prep = eval(item_dict['original_prep'])
    descriptions.append(original_prep)
    count += 1
    if count % 100000 == 0:
        print(count)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000


In [9]:
len(descriptions)

10902496

## Testing spacy

In [11]:
import spacy

In [12]:
model = spacy.load("pt_core_news_lg",
                   exclude=['parser','ner', 'lemmatizer', 'entity_linker', 'entity_ruler', 'textcat', 'textcat_multilabel', 'morphologizer', 'attribute_ruler', 'senter', 'sentencizer', 'tok2vec', 'transformer'],
                   disable=['parser','ner', 'lemmatizer', 'entity_linker', 'entity_ruler', 'textcat', 'textcat_multilabel', 'morphologizer', 'attribute_ruler', 'senter', 'sentencizer', 'tok2vec', 'transformer'])

In [14]:
start = time.time()
count = 0

for desc in descriptions:
    tags = []
    for token in desc:
        if token in word_class:
            tags.append(word_class[token])
    count += 1
    if count % 100000 == 0:
        print(count)
    
end = time.time()
(end - start)/60

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000


0.6071097532908122

In [16]:
start = time.time()
count = 0

for desc in descriptions:
    tags = []
    parser = model(' '.join(desc))
    for token in parser:
        tags.append(token.pos_)
    count += 1
    if count % 100000 == 0:
        print(count)
    
end = time.time()
(end - start)/60

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000


3.1455912152926127